In [ ]:

import glob

import rasterio
from shapely.geometry import LineString
import geopandas as gpd

import utils.visualization as viz
from utils.blobifier import blobify
from utils.vectorizer.vector_builder import VectorBuilder

from simplification.cutil import simplify_coords

! export PYTHONOPTIMIZE="" # show asserts










In [ ]:
input_filepath = glob.glob('data/sources/*.tif')[0]
output_filepath = "data/intermediates/vectors.shp"

In [ ]:
with rasterio.open(input_filepath) as src:
    meta = src.meta
    original = src.read(1)
    transform = src.transform

In [ ]:



min_blob_size = 30
cleaned = blobify(original, min_blob_size)

config = viz.get_show_config(original)
viz.show_raster(original, *config)
viz.show_raster(cleaned, *config)

In [ ]:
meters_per_pixel = 30
tolerance = meters_per_pixel

def simplify(segment):
    # Simplification will turn rings into what are effectively points.
    # We cut the ring in half to provide simplification with non-ring segments instead.
    if segment.is_ring:
        assert len(segment.coords) >= 3
        midpoint_idx = len(segment.coords) // 2
        segment1 = LineString(segment.coords[:midpoint_idx+1])
        segment2 = LineString(segment.coords[midpoint_idx:])
        simplified_coords1 = simplify_coords(segment1.coords, tolerance)
        simplified_coords2 = simplify_coords(segment2.coords, tolerance)
        simplified = LineString(simplified_coords1[:-1] + simplified_coords2)
    else:
        simplified_coords = simplify_coords(segment.coords, tolerance)
        simplified = LineString(simplified_coords)
    return simplified

vector_builder = VectorBuilder(cleaned, transform)
vector_builder.run_per_segment(simplify)
vector_builder.rebuild()
simplified_polygons, labels = vector_builder.get_result()

cmap = viz.generate_color_map(labels)
viz.show_polygons(simplified_polygons, labels, color_map=cmap)

In [ ]:
gdf = gpd.GeoDataFrame(geometry=simplified_polygons)
gdf['label'] = labels
gdf.crs = meta['crs']
gdf.to_file(output_filepath)